In [1]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*
import kotlinx.coroutines.runBlocking

val username = "Johann-25.0"
val intervalStart = LocalDate.parse("2019-01-01")
val intervalEnd = LocalDate.parse("2021-01-31")

val fundName = "Expenses"
val reportViewName = "Expenses report 1"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

val client = FundsClient()

In [2]:
val user = runBlocking { client.ensureUserExists(username) }
user

UserTO(id=2811ee93-fb16-43aa-9892-60058eec39b7, username=Johann-25.0)

In [3]:
val reportView = runBlocking {
    client.createReportView(user, reportViewName, fundName, File(REPORT_DATA_CONFIGURATION_YAML_FILE))
}
reportView

ReportViewTO(id=b41c49d3-bafa-4086-a54e-843f5e235688, name=Expenses report 1, fundId=ad7700a3-06f9-4fe7-8c20-2cacc77b611c, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]), forecast=ForecastReportFeatureTO(enabled=false, inputBuckets=0, outputBuckets=0))))

In [4]:
val monthlyReportData = runBlocking {
    client.getReportViewData(user, reportViewName, intervalStart, intervalEnd, TimeGranularity.MONTHLY)
}


In [ ]:
val yearlyData = runBlocking {
    client.getReportViewData(user, reportViewName, intervalStart, intervalEnd, TimeGranularity.YEARLY)
}

In [5]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotGroupData(reportData: ReportDataTO, plotTitle: String, groupItemFilter: (ReportDataGroupItemTO) -> Boolean): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "spent" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.spent!! }.reduce { acc, value -> acc + value }
        },
        "allocated" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! }.reduce { acc, value -> acc + value }
        },
        "left" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.left!! }.reduce { acc, value -> acc + value }
        },
        "net" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! + it.spent!! }.reduce { acc, value -> acc + value }
        },
    )
    return dataFrame
        .plot {
            line {
                val values = listOf("spent", "allocated", "left").flatMap { dataFrame.get(it).values }.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.granularInterval.granularity) {
                    TimeGranularity.YEARLY -> intervalEnd.minus(183, DateTimeUnit.DAY)
                    else -> intervalEnd.minus(15, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.granularInterval.granularity) {
                    TimeGranularity.YEARLY -> "%Y"
                    else -> "%b %Y"
                }
                axis.breaks(reportData.data.map { it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds() }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("spent")
                color = Color.RED
            }
            line {
                y("allocated")
                color = Color.GREEN
            }
            line {
                y("net")
                color = Color.YELLOW
            }
            area {
                y("left")
                borderLine {
                    color = Color.ORANGE
                }
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }
}

fun plotGroup(reportData: ReportDataTO, title: String, group: String): Plot =
    plotGroupData(reportData, title) { it.group == group }

fun plotGroupsTotal(reportData: ReportDataTO, title: String): Plot =
    plotGroupData(reportData, title) { true }



In [6]:
plotGroupsTotal(yearlyData, "Yearly total expenses")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="cXO1W0"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Yearly total expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12,1.7040672E12,1.7356896E12,1.7672256E12,1.7987616E12],
"left":[18160.866169342597,3544.0599255962934,1916.2533536346923,1600.1588677465706,1257.7231747011137,886.7511739018727,484.864839702685,49.4879776535748,-422.1702895663162],
"spent":[-103911.4574768,-186186.29,-15195.09,-25441.069789733334,-27561.15893887778,-29857.922183784256,-32346.082365766277,-35041.58922958014,-37961.721665378485],
"net":[12394.376169342599,-14588.42,-1599.09,-316.0944858881157,-342.4356930454572,-370.9720007992468,-401.886334199182,-435.376862049119,-471.658267219883],
"allocated":[116305.8336461426,171597.87,13596.0,25124.975303845218,27218.723245832323,29486.95018298501,31944.196031567095,34606.21236753102,37490.06339815861]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12,1.7040672E12,1.7356896E12,1.7672256E12,1.7987616E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12,1.7040672E12,1.7356896E12,1.7672256E12,1.7987616E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12,1.7040672E12,1.7356896E12,1.7672256E12,1.7987616E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12,1.7040672E12,1.7356896E12,1.7672256E12,1.7987616E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12,1.6725312E12,1.7040672E12,1.7356896E12,1.7672256E12,1.7987616E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[-186186.29,171597.87]
},
"sampling":"none",
"x":1.59624E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"net"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"left"
},
"stat":"identity",
"color":"#fc8452",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("cXO1W0");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height:

In [7]:
plotGroupsTotal(monthlyReportData, "Monthly total expenses")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Fjts4I"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Monthly total expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"left":[8248.009999999998,10951.57,17067.75,17292.506022686404,16409.800981436005,17634.7274648476,14404.39275595681,-784.3493119067911,4894.2030591987095,9818.25348725471,11803.609861628207,18160.866169342607,24395.160795337106,29941.272837978107,27811.898532850508,29842.07818091991,23411.57458146261,12509.842806606006,-4729.743404802091,-2051.7871160347945,-56.0184978675995,-320.4356971755895,2719.6396873483145,3544.0599255963166,1941.838989054615,73.5498223879418,-2413.55677483427,-4931.094755158341,-7827.767567176088,-10430.195613528651,-12340.723497077262],
"spent":[-4468.51,-4301.32,-9171.45,-6967.76,-9043.57,-7024.09,-11232.58,-25034.7199255,-4539.6390361,-6776.5,-9787.68,-5529.22,-4702.18,-5442.03,-35747.94,-17363.44,-18069.43,-20971.82,-27334.72,-15561.74,-8102.56,-10342.12,-8651.84,-13896.47,-15195.09,-16389.933333333338,-17302.25861111111,-15765.118495370367,-15631.925036651235,-15428.799623038838,-14966.881258292073],
"net":[2481.52,2703.56,6116.18,224.7560226864,-878.38,1231.8,-3232.75,-15187.9099255,5679.4044813615,4927.44,1994.64,6358.31,6231.96,5557.52,-2121.93,2032.08,-6427.61,-10905.23,-17239.74,2678.97,2008.06,-267.52,3042.74,822.28,-1599.09,-1868.2891666666692,-2487.1065972222195,-2517.53798032407,-2896.6728120177463,-2602.4280463525597,-1910.5278835486051],
"allocated":[6950.03,7004.88,15287.63,7192.5160226864,8165.19,8255.89,7999.83,9846.81,10219.0435174615,11703.94,11782.32,11887.53,10934.14,10999.55,33626.01,19395.52,11641.82,10066.59,10094.98,18240.71,10110.62,10074.6,11694.58,14718.75,13596.0,14521.644166666667,14815.15201388889,13247.580515046297,12735.252224633488,12826.371576686279,13056.353374743469]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,

In [8]:
plotGroup(monthlyReportData, "Basic Expenses", "basic")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="JlOi00"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"left":[1509.9159999999997,1102.0699999999997,2340.4390000000003,2060.4334863521926,2237.9913498352944,2166.0131301380543,616.6538920664033,781.4769679456256,1953.5977074267869,2624.9997067648073,2575.2376446975654,3029.270256716548,3277.070477189104,3529.9290142313957,8601.913567218762,11726.135009319567,12217.940199433944,11179.629739373604,10157.423175114578,9818.022184116258,9505.70176910789,7357.861792049072,7874.232836583473,7639.759515639494,7839.020306331671,8253.104181331671,8680.495795915005,8720.764861713616,8503.988916328779,8228.069792161872,8015.835415981055],
"spent":[-2305.04,-2509.31,-3347.92,-2293.91,-2108.18,-2382.69,-3709.61,-2493.78,-1587.87,-2253.23,-2992.86,-2517.62,-2158.1,-2165.51,-2324.86,-948.25,-1951.84,-3154.12,-3041.34,-3989.72,-2393.26,-3636.25,-1883.73,-3184.42,-2513.19,-2598.874166666667,-2634.987847222222,-2660.831834490741,-2803.546987364969,-2874.522569645383,-2851.222783782498],
"net":[-220.031,-407.846,1238.369,-280.005513647808,178.0732,-71.0408,-1549.6559,164.8587,1171.271749714605,672.755,-47.28,454.2625,247.4108,254.391,5072.8622,3124.8092,492.9422,-1040.1361,-1022.344,-341.578,-371.136,-1621.33,455.186,-240.67,206.01,414.083875,427.391614583334,40.269065798611,-216.775945384838,-275.919124166907,-212.234376180816],
"allocated":[2085.009,2101.464,4586.289,2013.904486352192,2286.2532,2311.6492,2159.9541,2658.6387,2759.141749714605,2925.985,2945.58,2971.8825,2405.5108,2419.901,7397.7222,4073.0592,2444.7822,2113.9839,2018.996,3648.142,2022.124,2014.92,2338.916,2943.75,2719.2,3012.958041666667,3062.379461805556,2701.100900289352,2586.771041980131,2598.603445478476,2638.988407601682]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.585

In [9]:
plotGroup(monthlyReportData, "Home Expenses", "home")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="4lWcUk"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Home Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"left":[3314.956,4807.26,7629.649,9531.25412725965,11741.731058362084,13911.46764007915,16293.313485323306,857.7557199312646,4012.112809380651,8000.051345551148,12069.76851652298,16185.95246461429,20161.689210379238,21815.46095972219,5562.230976955584,-2622.245883268029,-10715.5354205046,-23660.724666779603,-42910.00393770309,-45970.17250574397,-45562.96980984446,-41258.56888948625,-40020.28769550255,-37269.15442277844,-41808.66965290709,-47129.580152907096,-53032.49986124042,-58073.34571193486,-62852.25398352051,-67354.9426277383,-71154.22298397424],
"spent":[-500.0,-609.16,-1763.9,-400.0,-400.0,-467.23,-420.0,-18881.0,-423.24,-456.78,-400.0,-400.0,-400.0,-2736.62,-29698.21,-16136.26,-12866.69,-17070.89,-23488.88,-10712.07,-3563.87,-2752.85,-3681.19,-3840.81,-10690.64,-11436.58166666667,-12161.57847222222,-10700.19251157407,-10247.18688753858,-10028.89496150013,-9442.062041625139],
"net":[1585.009,1492.304,2822.389,1901.605127259648,2212.8608,2174.6548,2379.9405,-15434.6165,3153.425231111525,3990.7172,4077.2816,4117.2614,3973.656,1663.2,-16247.806,-8184.0968,-8093.5438,-12943.5881,-19248.9884,-3050.9718,682.5904,1780.72,1581.371,2782.6275,-4572.44,-5320.910500000003,-5902.919708333331,-5040.84585069444,-4778.908271585648,-4502.688644217787,-3799.280356235934],
"allocated":[2085.009,2101.464,4586.289,2301.605127259648,2612.8608,2641.8848,2799.9405,3446.3835,3576.665231111525,4447.4972,4477.2816,4517.2614,4373.656,4399.82,13450.404,7952.1632,4773.1462,4127.3019,4239.8916,7661.0982,4246.4604,4533.57,5262.561,6623.4375,6118.2,6115.671166666667,6258.658763888889,5659.34666087963,5468.278615952932,5526.206317282343,5642.781685389205]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E

In [10]:
plotGroup(monthlyReportData, "Transport Expenses", "transport")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="AqsEiX"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Transport Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"left":[431.586,573.98,124.24149999999997,345.9676215880479,362.8043913747458,592.3847247539878,944.4122590704538,257.7825003352964,744.1160343937266,1675.5746829048828,349.89234735217747,-481.79588706437113,112.38757683679441,1161.0855208116468,4629.1679398680235,6956.314034664291,7133.757924451113,8140.004671179787,8863.952139571995,11237.144222938865,12174.411763022446,12505.508983816631,11340.821009646903,9464.840742949438,11190.98919817717,12155.932839843837,13113.892576649392,13862.651366522077,14479.851522217486,15133.641824220846,15758.149584724484],
"spent":[-204.24,-207.85,-1214.12,-281.75,-554.64,-348.18,-288.04,-1474.32,-331.47,-238.42,-2502.33,-2020.41,-608.51,-161.2,-230.49,0.0,-1218.9,-202.81,-588.5,0.0,-444.43,-405.02,-2905.92,-3945.53,-304.89,-867.3075,-926.1497916666667,-984.1214409722222,-1066.131561053241,-1053.400857807677,-1124.283429291651],
"net":[143.2615,142.394,-449.7385,221.726121588048,16.9233,229.7323,351.9464,-686.5752,486.05348139692,931.974,-1324.098,-831.657,594.2454,1048.7505,3468.3711,2327.4624,178.1184,1005.1808,723.8474,2371.2923,869.9506,1005.424,-1268.6788,-1884.905,1734.51,964.943641666667,957.9597368055553,748.7587898726848,617.200155695409,653.79030200336,624.507760503639],
"allocated":[347.5015,350.244,764.3815,503.476121588048,571.5633,577.9123,639.9864,787.7448,817.52348139692,1170.394,1178.232,1188.753,1202.7554,1209.9505,3698.8611,2327.4624,1397.0184,1207.9908,1312.3474,2371.2923,1314.3806,1410.444,1637.2412,2060.625,2039.4,1832.251141666667,1884.109528472222,1732.880230844907,1683.33171674865,1707.191159811037,1748.79118979529]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.59356

In [11]:
plotGroup(monthlyReportData, "Shopping & Services Expenses", "shopping_services")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="P6RaQY"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Shopping & Services Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"left":[250.25599999999997,600.5,755.1115,-294.43753863881597,-741.4424966520149,-2740.7685311975124,-2644.33581289133,-1954.9063495023581,-1324.687096499022,-847.0638596533166,-21.500520487815592,810.6284988345645,1316.7911072102893,1976.14833357683,2733.4376033901626,3896.98203104808,4595.113365102493,5130.795380223344,5635.607455180325,6118.90081744503,6661.0875434491945,6695.121503265329,7141.578663842188,6842.828775902825,7307.58360084573,7829.879609179063,8340.702534873508,8830.945654375822,9265.064767169997,9677.151372697019,10078.995578684626],
"spent":[-385.57,0.0,-609.77,-1481.1,-936.99,-2494.99,-463.18,0.0,-82.99,-342.57,0.0,0.0,-149.99,0.0,-1259.78,0.0,0.0,-69.0,0.0,-429.95,0.0,0.0,-77.0,-893.12,-209.0,-244.8208333333333,-265.2225694444444,-182.3427835648148,-197.5380155285494,-213.9995168225952,-226.0828098911448],
"net":[-38.0685,350.244,154.6115,-1049.549038638816,-447.0786,-1999.6366,96.8081,689.2767,632.343046222305,476.7058,824.7624,832.1271,506.0584,659.973,757.7806,1163.7312,698.5092,534.9954,504.749,482.0855,505.531,402.984,390.7832,-304.37,470.8,522.2960083333334,510.8229256944445,490.2431195023148,434.1191127941744,412.0866055270222,401.8442059876074],
"allocated":[347.5015,350.244,764.3815,431.550961361184,489.9114,495.3534,559.9881,689.2767,715.333046222305,819.2758,824.7624,832.1271,656.0484,659.973,2017.5606,1163.7312,698.5092,603.9954,504.749,912.0355,505.531,402.984,467.7832,588.75,679.8,767.1168416666667,776.0454951388889,672.5859030671296,631.6571283227238,626.0861223496174,627.9270158787522]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1

In [12]:
plotGroup(monthlyReportData, "Fun Expenses", "fun")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ee6KoQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Fun Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"left":[876.3179999999996,1357.0499999999995,2412.3045000000006,1238.3315829492326,63.73656293693919,1136.9755658232368,1147.0242885006066,150.8190993601525,-583.1817364387047,-51.962643010463985,890.672090139836,1741.5949810471618,1818.8572675554774,2477.97971172908,3207.5442110985587,4177.101043650251,4758.787016318047,5262.827588028949,5666.691486111626,6047.53441052026,5869.187410807603,6047.8298155854545,6682.850156611042,7424.04676927121,8097.288127542461,8640.488044209127,9173.956870598016,9691.033049185979,10170.384242656273,10641.173785582423,11109.251665419088],
"spent":[-1031.16,-570.0,-1237.89,-2109.0,-2235.76,0.0,-790.09,-1980.8199255,-1756.0690361,-405.5,0.0,-100.0,-579.01,0.0,-1287.38,0.0,0.0,0.0,0.0,-350.0,-619.0,0.0,0.0,0.0,0.0,-188.0316666666667,-203.7009722222222,-113.3943865740741,-122.8439187885802,-133.0809120209619,-144.1709880227088],
"net":[11.3445,480.732,1055.2545,-1173.972917050768,-1174.2853,1073.2657,9.893,-996.1389255,-734.16468435385,530.8152,942.5856,851.0024,77.0384,659.973,730.1806,969.776,582.091,503.3295,403.7992,379.6284,-214.5752,503.73,584.729,735.9375,679.8,543.1999166666666,533.4688263888889,517.0761785879629,479.3511934702933,470.789542926151,468.0778798366636],
"allocated":[1042.5045,1050.732,2293.1445,935.027082949232,1061.4747,1073.2657,799.983,984.681,1021.90435174615,936.3152,942.5856,951.0024,656.0484,659.973,2017.5606,969.776,582.091,503.3295,403.7992,729.6284,404.4248,503.73,584.729,735.9375,679.8,731.2315833333333,737.1697986111111,630.470565162037,602.1951122588735,603.8704549471129,612.2488678593724]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12

In [13]:
plotGroup(monthlyReportData, "Gifts Expenses", "gifts")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="hspd6i"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Gifts Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"left":[1525.9824,1974.5680000000002,3661.2336,4122.335522722368,3171.127282388459,2830.505803270139,-1850.7852244216385,-972.3292044828909,-200.29074753497173,833.2025675715074,-1999.6664650830899,-1420.8002575627015,-1094.1389050853122,-323.87225074211074,2896.8892736854887,4637.963332897036,3796.695618634454,4328.921931867861,5122.4731490680015,6867.642032848577,6837.845148248435,6777.072705976606,7664.994540075155,8004.936557684583,7711.254739230997,8468.422605897664,9224.541544786553,9775.280311916182,10226.807976306614,10786.031112729583,11347.551260521132],
"spent":[0.0,-392.0,-147.85,-402.0,-1930.0,-1330.0,-5561.66,-204.8,-351.0,-20.0,-3892.49,-491.19,-766.57,-330.2,-141.92,-198.28,-2005.0,-475.0,-216.0,-80.0,-1082.0,-488.0,-104.0,-843.59,-1102.37,-588.8633333333333,-610.4186111111111,-649.460162037037,-687.0585088734568,-577.2300512795782,-585.749222219543],
"net":[834.0036,448.5856,1686.6656,461.101922722368,-950.1772,-339.2932,-4681.6787,878.3491,773.094786920765,1033.3546,-2832.0812,578.6877,326.844,769.755,3220.681,1741.272,-840.818,531.659,793.498,1744.071,-70.938,317.968,831.5664,333.91,-286.61,757.1678666666667,756.1189388888889,550.73876712963,451.5276643904322,559.2231364229679,561.520147791549],
"allocated":[834.0036,840.5856,1834.5156,863.101922722368,979.8228,990.7068,879.9813,1083.1491,1124.094786920765,1053.3546,1060.4088,1069.8777,1093.414,1099.955,3362.601,1939.552,1164.182,1006.659,1009.498,1824.071,1011.062,805.968,935.5664,1177.5,815.76,1346.0312,1366.53755,1200.198929166667,1138.586173263889,1136.453187702546,1147.269370011092]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.59891

In [14]:
plotGroup(monthlyReportData, "Development Expenses", "development")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="oPoeqz"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Development Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"left":[338.99559999999997,536.142,144.77090000000004,288.621220453728,-426.14716680950426,-261.8508680194531,-101.89013169099127,95.05195450611879,292.53608847024327,-2416.5483128738556,-2060.793751513447,-1703.9838872428859,-1197.4959387484841,-695.4584513509252,180.7149606339273,1069.8286126087132,1624.8158780271579,2128.3881627120636,2734.1131278544735,3829.1417218401857,4458.717677341292,1554.738391617568,2035.4501760921037,1436.8019869272068,1604.372669833676,1855.3026948336758,2085.353763583676,2261.5757130628426,2378.3889916652734,2458.6791268179063,2503.7159815665923],
"spent":[-42.5,-13.0,-850.0,0.0,-878.0,-1.0,0.0,0.0,-7.0,-3060.0,0.0,0.0,-40.0,-48.5,-805.3,-80.65,-27.0,0.0,0.0,0.0,0.0,-3060.0,0.0,-1189.0,-375.0,-465.4541666666667,-500.2003472222222,-474.7753761574074,-507.619157503858,-547.6707539625129,-593.309983459389],
"net":[166.0009,197.1464,-391.3711,143.850320453728,-714.6962,164.1178,159.9966,196.9362,197.38087034923,-2708.8818,353.4696,356.6259,506.707,501.4775,876.0005,889.126,555.091,503.3295,605.6988,1094.4426,606.6372,-2657.016,467.7832,-600.25,168.84,250.930025,230.05106875,176.2219494791667,116.8132786024306,80.2901351526331,45.0368547486858],
"allocated":[208.5009,210.1464,458.6289,143.850320453728,163.3038,165.1178,159.9966,196.9362,204.38087034923,351.1182,353.4696,356.6259,546.707,549.9775,1681.3005,969.776,582.091,503.3295,605.6988,1094.4426,606.6372,402.984,467.7832,588.75,543.84,716.3841916666667,730.2514159722222,650.9973256365741,624.4324361062886,627.960889115146,638.3468382080748]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,

In [15]:
plotGroup(monthlyReportData, "Investment Expenses", "investment")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="hBizLz"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Investment Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"left":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"spent":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"net":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"allocated":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616